In [2]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
import joblib

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/pplonski/datasets-for-start/master/adult/data.csv', skipinitialspace=True)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
X = df.drop(['income'], axis=1)
y = df.income

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [56]:
train_mode = dict(X_train.mode().iloc[0])
X_train = X_train.fillna(train_mode)
print(train_mode)

{'age': 31.0, 'workclass': 'Private', 'fnlwgt': 121124, 'education': 'HS-grad', 'education-num': 9.0, 'marital-status': 'Married-civ-spouse', 'occupation': 'Prof-specialty', 'relationship': 'Husband', 'race': 'White', 'sex': 'Male', 'capital-gain': 0.0, 'capital-loss': 0.0, 'hours-per-week': 40.0, 'native-country': 'United-States'}


In [57]:
encoders = {}

for column in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race',
              'sex', 'native-country']:
    categorical_convert = LabelEncoder()
    X_train[column] = categorical_convert.fit_transform(X_train[column])
    encoders[column] = categorical_convert

In [58]:
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)

In [17]:
et = ExtraTreesClassifier(n_estimators=100)
et = et.fit(X_train, y_train)

In [18]:
"""joblib.dump(train_mode, "./train_mode.joblib", compress=True)
joblib.dump(encoders, "./encoders.joblib", compress=True)
joblib.dump(rf, "./random_forest.joblib", compress=True)
joblib.dump(et, "./extra_trees.joblib", compress=True)"""

['./extra_trees.joblib']

In [68]:
X_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
29700,37,3,34146,11,9,2,2,0,4,1,0,0,68,37
1529,37,3,26898,11,9,0,3,4,4,0,0,0,12,37
27477,26,3,190762,15,10,2,11,0,4,1,0,0,18,37
31950,35,3,189092,9,13,2,3,0,4,1,0,0,40,37
4732,23,3,260019,5,4,4,4,4,3,1,0,0,36,24


In [70]:
X_test = X_test.fillna(train_mode)

for column in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race',
              'sex']:
    cc = encoders[column]
    X_test[column] = cc.transform(X_test[column])

In [61]:
X_test = X_test.replace('Holand-Netherlands', 'United-States')

In [60]:
X_test['native-country'].value_counts()

United-States                 8731
Mexico                         163
Philippines                     70
Germany                         43
Cuba                            36
Jamaica                         36
Canada                          35
England                         35
El-Salvador                     31
Italy                           30
India                           29
Puerto-Rico                     25
Vietnam                         25
Dominican-Republic              23
Columbia                        23
Guatemala                       22
China                           20
Japan                           19
South                           19
Taiwan                          16
Haiti                           16
Poland                          14
Nicaragua                       14
Peru                            13
Iran                            11
Portugal                        10
Ecuador                         10
Greece                           9
Ireland             

In [71]:
rf.score(X_test, y_test)

0.8516736615825571

In [74]:
input_data = {"age": 37,
            "workclass": "Private",
            "fnlwgt": 34146,
            "education": "HS-grad",
            "education-num": 9,
            "marital-status": "Married-civ-spouse",
            "occupation": "Craft-repair",
            "relationship": "Husband",
            "race": "White",
            "sex": "Male",
            "capital-gain": 0,
            "capital-loss": 0,
            "hours-per-week": 68,
            "native-country": "United-States"}

In [75]:
input_data = pd.DataFrame(input_data, index=[0])

for column in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race',
              'sex', 'native-country']:
    cc = encoders[column]
    input_data[column] = cc.transform(input_data[column])

In [76]:
input_data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,37,3,34146,11,9,2,2,0,4,1,0,0,68,37


In [98]:
rf.predict(input_data)

array(['<=50K'], dtype=object)

In [97]:
mapping = dict(zip(encoders['income'].classes_, range(len(encoders['income'].classes_))))

KeyError: 'income'

In [96]:
mapping

{'Female': 0, 'Male': 1}

In [87]:
from collections import defaultdict
d = defaultdict(categorical_convert)

TypeError: first argument must be callable or None